# 1. Setup

## 1.1 Module import

Import necessary modules

In [18]:
import tensorflow as tf
import keras
import dataset_utils

## 1.2 Dataset import using opencv

Define parameters for the image import

In [19]:
img_width = 100
img_height = 40

train_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped'
validation_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped'

Import the data for the training dataset

In [20]:
train_labels, train_images = dataset_utils.load_dataset(path_to_dataset = train_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 18777 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 18777 images and 18777 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped
Loaded images have shape (40, 100, 3).


Import the data for the validation dataset

In [21]:
validation_labels, validation_images = dataset_utils.load_dataset(path_to_dataset = validation_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 1812 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 1812 images and 1812 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped
Loaded images have shape (40, 100, 3).


# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [22]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)

base_model = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


base_model.trainable = False

(40, 100, 3)


Add the custom classification layer to it

In [23]:
# new layers
global_average_layer = tf.keras.layers.Flatten()
dense_layer_1 = tf.keras.layers.Dense(32, activation='elu')
prediction_layer = tf.keras.layers.Dense(3, activation = 'softmax')


# construct the final model
inputs = keras.Input(shape = (img_height, img_width, 3))
x = keras.applications.mobilenet_v3.preprocess_input(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = dense_layer_1(x)
#x = keras.layers.Dropout(0.05)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 40, 100, 3)]      0         
                                                                 
 MobilenetV3small (Function  (None, 2, 4, 576)         939120    
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 32)                147488    
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 1086707 (4.15 MB)
Trainable params: 147587 (576.51 KB)
Non-trainable params: 939120 (3.58 MB)
_________________

In [24]:
model.layers[-3].output_shape

(None, 4608)

## 2.2 Start the initial learning process

Define Parameters for the learning process

In [25]:
base_learning_rate = 0.03
initial_epochs = 30

Compile the before built model

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [27]:
def scheduler(epoch, lr):
    if epoch == 0:
        return lr
    elif epoch % 5 == 0:
        return lr * 0.9 
    else: 
        return lr

In [28]:
#callback = keras.callbacks.LearningRateScheduler(scheduler)
callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

In [29]:
generator = keras.preprocessing.image.ImageDataGenerator()
batch_size = 32

history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs, callbacks=callback)

Epoch 1/30


2024-05-23 04:51:37.208123: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-23 04:51:39.127299: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9f211bfc00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-23 04:51:39.127362: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 4GB Laptop GPU, Compute Capability 8.6
2024-05-23 04:51:39.182800: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1716432699.401473   10437 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


586/586 [==============================] - 22s 23ms/step - loss: 0.6766 - accuracy: 0.8785 - val_loss: 0.1656 - val_accuracy: 0.9404 - lr: 0.0300
Epoch 2/30
586/586 [==============================] - 12s 20ms/step - loss: 0.1955 - accuracy: 0.9324 - val_loss: 0.1387 - val_accuracy: 0.9547 - lr: 0.0300
Epoch 3/30
586/586 [==============================] - 12s 20ms/step - loss: 0.1785 - accuracy: 0.9358 - val_loss: 0.2245 - val_accuracy: 0.9205 - lr: 0.0300
Epoch 4/30
586/586 [==============================] - 12s 20ms/step - loss: 0.4016 - accuracy: 0.8234 - val_loss: 0.4110 - val_accuracy: 0.6004 - lr: 0.0300
Epoch 5/30
586/586 [==============================] - 12s 21ms/step - loss: 0.2710 - accuracy: 0.9197 - val_loss: 0.2000 - val_accuracy: 0.9487 - lr: 0.0300
Epoch 6/30
586/586 [==============================] - 12s 20ms/step - loss: 0.2718 - accuracy: 0.9304 - val_loss: 0.3120 - val_accuracy: 0.9415 - lr: 0.0300
Epoch 7/30
586/586 [==============================] - 12s 20ms/step -

In [ ]:
model.evaluate(x = validation_images, y = validation_labels)

# 3. Fine tuning the base model

Freeze all layers except for the uppermost layers of the base model. Layzers will be frozen from ***fine_tune_from*** to ***len(base_model.layers)***.

In [30]:
base_model.trainable = True
fine_tune_from = len(base_model.layers) - 25

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

# don't train the dense layer
model.layers[len(model.layers) - 1].trainable = False
model.layers[len(model.layers) - 2].trainable = False

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 40, 100, 3)]      0         
                                                                 
 MobilenetV3small (Function  (None, 2, 4, 576)         939120    
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 32)                147488    
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 1086707 (4.15 MB)
Trainable params: 294096 (1.12 MB)
Non-trainable params: 792611 (3.02 MB)
___________________

Compile the model again but decrease the learning rate in order to avoid overfitting.

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100), loss='categorical_crossentropy', metrics=['accuracy'])

Train the uppermost layers of the base model and proceed where you left off in the initial training by using the last epoch of the first training as the initial epoch for fine tuning.

In [32]:
finetune_history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=generator.flow(validation_images, validation_labels, batch_size), initial_epoch=history.epoch[-1], epochs=initial_epochs + 10, callbacks=callback)

Epoch 30/40
586/586 [==============================] - 34s 43ms/step - loss: 0.1037 - accuracy: 0.9701 - val_loss: 0.2718 - val_accuracy: 0.9272 - lr: 3.0000e-04
Epoch 31/40
586/586 [==============================] - 17s 29ms/step - loss: 0.0933 - accuracy: 0.9727 - val_loss: 0.1899 - val_accuracy: 0.9608 - lr: 3.0000e-04
Epoch 32/40
586/586 [==============================] - 16s 27ms/step - loss: 0.0837 - accuracy: 0.9761 - val_loss: 0.1682 - val_accuracy: 0.9636 - lr: 3.0000e-04
Epoch 33/40
586/586 [==============================] - 16s 27ms/step - loss: 0.0739 - accuracy: 0.9772 - val_loss: 0.1895 - val_accuracy: 0.9520 - lr: 3.0000e-04
Epoch 34/40
586/586 [==============================] - 17s 29ms/step - loss: 0.0696 - accuracy: 0.9791 - val_loss: 0.1532 - val_accuracy: 0.9647 - lr: 3.0000e-04
Epoch 35/40
586/586 [==============================] - 16s 28ms/step - loss: 0.0634 - accuracy: 0.9811 - val_loss: 0.1880 - val_accuracy: 0.9547 - lr: 3.0000e-04
Epoch 36/40
586/586 [=======

: 

In [ ]:
print(finetune_history.history.keys())
key = 'accuracy'
for loss in history.history[key]:
    print(loss, end='\r\n')
for loss in finetune_history.history[key]:
    print(loss, end='\r\n')

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [ ]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/mn3_w100_h40_crop_14_contrast_grayscale_96_val_99_train.keras'

model.save(save_path)